# DPO (Direct Preference Optimization) 实战教程

这个notebook将带你学习DPO算法，一种比RLHF更简单高效的偏好优化方法

## 1. DPO算法核心概念

### DPO的关键优势：
- 🚀 **简化流程**: 只需2步 (SFT → DPO)，而不是3步 (SFT → RM → PPO)
- 📈 **训练稳定**: 避免PPO的不稳定性问题
- 💰 **降低成本**: 无需训练奖励模型
- 🎯 **效果相当**: 在多个基准上与RLHF性能相当

In [ ]:
# 导入必要的库
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import DPOTrainer
from datasets import Dataset
import numpy as np
import matplotlib.pyplot as plt

print("📚 库导入完成")
print(f"PyTorch版本: {torch.__version__}")

## 2. 准备偏好数据

DPO需要偏好对比数据，每个样本包含：
- `prompt`: 用户提示
- `chosen`: 更好的回复
- `rejected`: 较差的回复

In [ ]:
def create_preference_data():
    """创建高质量的偏好对比数据"""
    
    preference_pairs = [
        {
            "prompt": "请解释什么是人工智能",
            "chosen": "人工智能(AI)是计算机科学的一个分支，旨在创建能够执行通常需要人类智能的任务的系统，如学习、推理、感知和决策。",
            "rejected": "人工智能就是人工的智能，很智能的人工，智能人工。"
        },
        {
            "prompt": "给我一些学习编程的建议",
            "chosen": "学习编程建议：1)选择合适的语言开始；2)多做实际项目练习；3)阅读他人优秀代码；4)加入编程社区交流；5)保持持续学习的习惯。",
            "rejected": "编程就是编程，学就是学，编程学习学习编程。"
        },
        {
            "prompt": "推荐一个周末活动",
            "chosen": "推荐去公园散步或骑行，既能锻炼身体又能享受自然风光，如果天气不好可以在家读书或学习新技能。",
            "rejected": "周末就是周末活动活动活动活动。"
        },
        {
            "prompt": "如何提高写作能力",
            "chosen": "提高写作能力需要：多读优秀文章积累素材，每天坚持写作练习，学习不同的写作技巧，请他人给予反馈建议。",
            "rejected": "写作写作写作写作写作写作写作。"
        },
        {
            "prompt": "描述一下理想的假期",
            "chosen": "理想的假期是和亲朋好友一起，去一个风景优美的地方，放慢节奏，享受当下，充电休息后以更好的状态投入工作。",
            "rejected": "假期假期假期假期假期假期假期。"
        }
    ] * 15  # 创建更多样本
    
    return preference_pairs

# 创建数据集
preference_data = create_preference_data()
print(f"偏好数据集大小: {len(preference_data)}")
print(f"\n示例数据:")
print(f"提示: {preference_data[0]['prompt']}")
print(f"好回复: {preference_data[0]['chosen'][:50]}...")
print(f"差回复: {preference_data[0]['rejected'][:50]}...")

## 3. 设置DPO训练器

In [ ]:
# 加载模型和tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 设置pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"模型加载完成: {model_name}")
print(f"词汇表大小: {len(tokenizer)}")

In [ ]:
# 准备数据集
train_dataset = Dataset.from_list(preference_data)

print(f"训练数据集创建完成，大小: {len(train_dataset)}")
print("\n数据集字段:", train_dataset.column_names)

In [ ]:
# 训练参数设置
training_args = TrainingArguments(
    output_dir="./models/dpo_gpt2",
    num_train_epochs=2,
    per_device_train_batch_size=1,  # 小批次避免内存问题
    gradient_accumulation_steps=4,
    warmup_steps=5,
    logging_steps=5,
    save_steps=20,
    save_strategy="epoch",
    evaluation_strategy="no",
    report_to=None,
    remove_unused_columns=False,
    dataloader_drop_last=True,
)

print("训练参数设置完成")

## 4. 开始DPO训练

In [ ]:
# 创建DPO训练器
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # 将自动创建参考模型
    args=training_args,
    beta=0.1,  # DPO温度参数，控制与参考模型的偏差
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    max_length=128,
    max_prompt_length=64,
)

print("✅ DPO训练器创建成功")
print(f"Beta参数: {dpo_trainer.beta}")
print(f"最大长度: {dpo_trainer.max_length}")

In [ ]:
# 开始训练
print("🚀 开始DPO训练...")

# 训练模型
dpo_trainer.train()

print("✅ DPO训练完成！")

## 5. 保存和测试模型

In [ ]:
# 保存模型
dpo_trainer.save_model()
tokenizer.save_pretrained("./models/dpo_gpt2")

print("💾 模型已保存到 ./models/dpo_gpt2")

In [ ]:
# 测试训练后的模型
def generate_with_model(model, tokenizer, prompt, max_length=100):
    """使用模型生成文本"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 测试几个例子
test_prompts = [
    "请解释什么是人工智能",
    "给我一些学习编程的建议", 
    "如何提高写作能力"
]

print("🧪 测试DPO训练后的模型:")
print("=" * 50)

for prompt in test_prompts:
    response = generate_with_model(model, tokenizer, prompt)
    generated_part = response[len(prompt):].strip()
    
    print(f"\n📝 提示: {prompt}")
    print(f"🤖 回复: {generated_part}")
    print("-" * 30)

## 6. 深入理解DPO

### DPO损失函数可视化

In [ ]:
# 可视化DPO损失函数
import matplotlib.pyplot as plt
import numpy as np

def dpo_loss(logits_diff, beta=0.1):
    """DPO损失函数"""
    return -torch.log(torch.sigmoid(beta * logits_diff))

# 创建数据点
x = np.linspace(-5, 5, 100)
betas = [0.1, 0.5, 1.0, 2.0]

plt.figure(figsize=(10, 6))

for beta in betas:
    y = [-np.log(1/(1 + np.exp(-beta * xi))) for xi in x]
    plt.plot(x, y, label=f'β={beta}')

plt.xlabel('log π(chosen) - log π(rejected)')
plt.ylabel('DPO Loss')
plt.title('DPO损失函数 vs β参数')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("📊 DPO损失函数可视化完成")
print("💡 解读: β值越大，对偏好差异越敏感")

## 7. 模型对比实验

In [ ]:
# 比较原始模型和DPO模型
def compare_models():
    """比较原始模型和DPO训练后模型的输出质量"""
    
    # 加载原始模型
    original_model = AutoModelForCausalLM.from_pretrained("gpt2")
    original_tokenizer = AutoTokenizer.from_pretrained("gpt2")
    if original_tokenizer.pad_token is None:
        original_tokenizer.pad_token = original_tokenizer.eos_token
    
    test_prompts = [
        "给我一个健康生活的建议",
        "解释什么是深度学习",
        "推荐一本好书"
    ]
    
    print("🔍 模型对比实验")
    print("=" * 60)
    
    for i, prompt in enumerate(test_prompts):
        print(f"\n🎯 测试 {i+1}: {prompt}")
        print("-" * 40)
        
        # 原始模型输出
        original_output = generate_with_model(original_model, original_tokenizer, prompt)
        original_response = original_output[len(prompt):].strip()
        
        # DPO模型输出
        dpo_output = generate_with_model(model, tokenizer, prompt)
        dpo_response = dpo_output[len(prompt):].strip()
        
        print(f"📖 原始GPT2: {original_response}")
        print(f"🎯 DPO训练后: {dpo_response}")

# 运行对比
compare_models()

## 8. DPO vs PPO 性能分析

In [ ]:
# 分析DPO的优势
comparison_data = {
    '训练阶段': ['PPO (RLHF)', 'DPO'],
    '步骤数': [3, 2],
    '需要奖励模型': ['是', '否'],
    '训练稳定性': ['中等', '高'],
    '实现复杂度': ['高', '低'],
    '计算开销': ['高', '中等']
}

import pandas as pd

df = pd.DataFrame(comparison_data)
print("📊 DPO vs PPO 对比表:")
print(df.to_string(index=False))

# 绘制对比图
metrics = ['简化程度', '稳定性', '效率', '易用性']
ppo_scores = [6, 7, 6, 5]
dpo_scores = [9, 9, 8, 9]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, ppo_scores, width, label='PPO (RLHF)', alpha=0.8)
ax.bar(x + width/2, dpo_scores, width, label='DPO', alpha=0.8)

ax.set_ylabel('评分')
ax.set_title('DPO vs PPO 综合评估')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim(0, 10)

plt.tight_layout()
plt.show()

print("📈 DPO在多个维度上都优于传统RLHF")

## 9. 交互式测试

In [ ]:
# 交互式测试DPO模型
def interactive_test():
    """交互式测试DPO模型"""
    
    print("🎮 DPO模型交互测试")
    print("输入你的问题，模型会给出回答")
    print("(在notebook中运行此代码需要交互环境)")
    
    # 这里提供几个测试样例
    test_questions = [
        "如何学习机器学习",
        "推荐一个编程项目",
        "给我一些时间管理建议"
    ]
    
    for question in test_questions:
        print(f"\n❓ 问题: {question}")
        response = generate_with_model(model, tokenizer, question)
        answer = response[len(question):].strip()
        print(f"🤖 DPO模型回答: {answer}")

# 运行交互测试
interactive_test()

## 10. 总结和下一步

通过这个教程，你已经学会了：

✅ **DPO的核心概念**：直接偏好优化，无需奖励模型  
✅ **数据准备**：如何构建偏好对比数据集  
✅ **模型训练**：使用TRL进行DPO训练  
✅ **效果评估**：比较训练前后的模型表现  

### 下一步学习方向：
1. 🔧 **高级定制**：自定义奖励函数和训练策略
2. 🏭 **生产部署**：模型优化和部署最佳实践
3. 📊 **评估指标**：更全面的模型评估方法
4. 🎯 **实际应用**：将DPO应用到具体任务中